In [1]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, OneHotEncoderEstimator, StringIndexer
from pyspark.sql import SparkSession, DataFrame, functions as F
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline, Transformer, PipelineModel
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable, DefaultParamsReader 
from pyspark.ml.param.shared import HasOutputCols, Param, Params, TypeConverters
from pyspark import keyword_only

In [2]:
spark.conf.set(
  "fs.azure.account.key.<storage-account-name>.blob.core.windows.net",
  "<account-key>")

In [3]:
data = spark.read.csv("wasbs://data@sparkmltrainig.blob.core.windows.net/train.csv", inferSchema = True, header = True)

train_df, test_df = data.randomSplit([0.9, 0.1], seed=42)

800
91

In [4]:
train_df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in train_df.columns]).show()
test_df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in test_df.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
 0| 0| 0| 0| 0|162| 0| 0| 0| 0| 618| 2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
 0| 0| 0| 0| 0| 15| 0| 0| 0| 0| 69| 0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+

In [5]:
# first way of data engineering - methods
def evaluate_initials(df: DataFrame) -> DataFrame:
  dizip_initials = {k:v for k,v in (zip(['Mlle','Mme','Ms','Dr', 'Major','Lady','Countess', 'Jonkheer','Col','Rev', 'Capt','Sir','Don'], ['Miss','Miss','Miss', 'Mr','Mr','Mrs','Mrs', 'Other','Other','Other', 'Mr','Mr','Mr']))}
  _df = df.withColumn('Initial',  F.regexp_extract( df['Name'], ('([A-Za-z]+)\.'),1 ) )
  _df = _df.replace(dizip_initials,1,'Initial')
  return _df
  
def handle_missing_age(df: DataFrame) -> DataFrame:
    _df = df
    _df = _df.withColumn('Age', 
           F.when((F.isnull(_df['Age'])) & (_df['Initial'] == 'Mr') , 33 )\
            .otherwise(F.when((F.isnull(_df['Age'])) 
                              & (_df['Initial'] == 'Mrs') , 36)\
            .otherwise(F.when((F.isnull(_df['Age'])) 
                              & (_df['Initial'] == 'Master') , 5)\
            .otherwise(F.when((F.isnull(_df['Age'])) 
                              & (_df['Initial'] == 'Miss') , 22)\
            .otherwise(F.when((F.isnull(_df['Age'])) 
                              & (_df['Initial'] == 'Other') , 46)\
            .otherwise(_df['Age']) )))))
    return _df

def create_family_size(df: DataFrame) -> DataFrame:
  _df = df.withColumn('FamilySize', df['Parch'] + df['SibSp'] + 1 )
  return _df

def create_is_alone(df: DataFrame) -> DataFrame:
  _df = df.withColumn('IsAlone', F.when(df['FamilySize'] > 1, 0).otherwise(1))
  return _df

def create_fare_per_person(df: DataFrame) -> DataFrame:  
  _df = df.withColumn('FarePerPerson', df['Fare'] /df['FamilySize'])
  return _df

def drop_rows_with_null(df: DataFrame, col) -> DataFrame:
  _df = df = df.filter(df[col].isNotNull())
  return _df

def change_to_index(df: DataFrame, col) -> DataFrame:
  indexer = StringIndexer(inputCol=col, outputCol='{0}_indexed'.format(col))
  _df = indexer.fit(df).transform(df)
  return _df

def change_to_one_hot_encoded(df: DataFrame, cols) -> DataFrame:
  for col in cols:
    df = change_to_index(df, col)
  col_indexed = ['{0}_indexed'.format(col) for col in cols]
  col_encoded = ['{0}_encoded'.format(col) for col in cols]
  encoder = OneHotEncoderEstimator(inputCols=col_indexed, outputCols=col_encoded)
  _df = encoder.fit(df).transform(df)
  return _df

<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
<command-2671224804740862>:3: DeprecationWarning: invalid escape sequence \.
 _df = df.withColumn('Initial', F.regexp_extract( df['Name'], ('([A-Za-z]+)\.'),1 ) )

In [6]:
# data engineering with method 1
train_df = drop_rows_with_null(handle_missing_age(evaluate_initials(create_fare_per_person(create_is_alone(create_family_size(train_df))))), 'Embarked')
test_df = handle_missing_age(evaluate_initials(create_fare_per_person(create_is_alone(create_family_size(test_df)))))
train_df = change_to_one_hot_encoded(train_df, ['Sex', 'Initial', 'Embarked'])
test_df = change_to_one_hot_encoded(test_df, ['Sex', 'Initial', 'Embarked'])
cols_to_drop=['PassengerId', 'Cabin', 'Ticket', 'Name', 'Sex', 'Initial', 'Embarked']
train_df = train_df.drop(*cols_to_drop)
test_df = test_df.drop(*cols_to_drop)

In [7]:
# learning the model
cols = [c for c in train_df.columns if c != 'Survived']
vec_assembler = VectorAssembler(inputCols=cols, outputCol="features")
vec_train_DF = vec_assembler.transform(train_df)
lr = LinearRegression(featuresCol="features", labelCol="Survived")
lr_model = lr.fit(vec_train_DF)

In [8]:
# applying model to test data
vec_test_df = vec_assembler.transform(test_df)
pred_df = lr_model.transform(vec_test_df)

+--------+------+----+-----+-----+-------+----------+-------+-----------------+-----------+---------------+----------------+-------------+---------------+----------------+--------------------+-------------------+
Survived|Pclass| Age|SibSp|Parch| Fare|FamilySize|IsAlone| FarePerPerson|Sex_indexed|Initial_indexed|Embarked_indexed| Sex_encoded|Initial_encoded|Embarked_encoded| features| prediction|
+--------+------+----+-----+-----+-------+----------+-------+-----------------+-----------+---------------+----------------+-------------+---------------+----------------+--------------------+-------------------+
 1| 3|26.0| 0| 0| 7.925| 1| 1| 7.925| 1.0| 1.0| 0.0| (1,[],[])| (4,[1],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.5905510743011597|
 0| 3| 2.0| 4| 1| 29.125| 6| 0|4.854166666666667| 0.0| 3.0| 2.0|(1,[0],[1.0])| (4,[3],[1.0])| (2,[],[])|[3.0,2.0,4.0,1.0,...| 0.5026026381839436|
 0| 1|42.0| 1| 0| 52.0| 2| 0| 26.0| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,2,4,5,7,...|0.34593756409002596|
 0| 3| 7.0| 4| 1|39.6875| 6| 0|6.614583333333333| 0.0| 3.0| 0.0|(1,[0],[1.0])| (4,[3],[1.0])| (2,[0],[1.0])|[3.0,7.0,4.0,1.0,...| 0.4099846281289141|
 0| 3|21.0| 0| 0| 7.8| 1| 1| 7.8| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.1104816241118205|
 0| 2|34.0| 1| 0| 26.0| 2| 0| 13.0| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,2,4,5,7,...|0.20233258877954152|
 0| 3|16.0| 0| 0| 9.2167| 1| 1| 9.2167| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...|0.12901409255927887|
 1| 3|22.0| 0| 0| 7.75| 1| 1| 7.75| 1.0| 1.0| 0.0| (1,[],[])| (4,[1],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.6049391507353366|
 0| 3|21.0| 0| 0| 7.925| 1| 1| 7.925| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.110523460678571|
 1| 1|44.0| 0| 0|27.7208| 1| 1| 27.7208| 1.0| 2.0| 1.0| (1,[],[])| (4,[2],[1.0])| (2,[1],[1.0])|[1.0,44.0,0.0,0.0...| 1.0018375122637517|
 1| 3| 5.0| 4| 2|31.3875| 7| 0|4.483928571428572| 1.0| 1.0| 0.0| (1,[],[])| (4,[1],[1.0])| (2,[0],[1.0])|[3.0,5.0,4.0,2.0,...| 0.2912286516482445|
 0| 2|24.0| 0| 0| 10.5| 1| 1| 10.5| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.260365813786285|
 1| 3|29.0| 0| 2|15.2458| 3| 0|5.081933333333333| 1.0| 2.0| 1.0| (1,[],[])| (4,[2],[1.0])| (2,[1],[1.0])|[3.0,29.0,0.0,2.0...| 0.6513212744347145|
 0| 3|33.0| 0| 0| 7.75| 1| 1| 7.75| 0.0| 0.0| 2.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[],[])|(18,[0,1,4,5,6,7,...| 0.1479151426729084|
 0| 2|36.0| 0| 0| 10.5| 1| 1| 10.5| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.2170258709034023|
 1| 3|25.0| 0| 0| 0.0| 1| 1| 0.0| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,5,6,11,1...|0.09342437471895959|
 0| 3| 7.0| 4| 1| 29.125| 6| 0|4.854166666666667| 0.0| 3.0| 2.0|(1,[0],[1.0])| (4,[3],[1.0])| (2,[],[])|[3.0,7.0,4.0,1.0,...| 0.4845443286494092|
 1| 1|26.0| 0| 0| 78.85| 1| 1| 78.85| 1.0| 1.0| 0.0| (1,[],[])| (4,[1],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.9339201533821605|
 0| 3|24.0| 0| 0| 8.85| 1| 1| 8.85| 1.0| 1.0| 0.0| (1,[],[])| (4,[1],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.5980839887089275|
 0| 3|22.0| 0| 0| 7.25| 1| 1| 7.25| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...|0.10668588131121126|
+--------+------+----+-----+-----+-------+----------+-------+-----------------+-----------+---------------+----------------+-------------+---------------+----------------+--------------------+-------------------+
only showing top 20 rows

In [9]:
# evaluating the model
regression_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Survived", metricName="rmse")

rmse = regression_evaluator.evaluate(pred_df)
print(f"RMSE is {rmse}")

RMSE is 0.34149375724898745

In [10]:
# second way of data engineering - custom Transformators
# hint: transform method should contain the same code as in the analogous method above

class InitialsTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable, DefaultParamsReader):
  # TODO implement a transformer following the instructions from https://akisiel1.github.io/introSparkML/#create-custom-pipeline-stage

class MissingAgeTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
 # TODO implement a transformer following the instructions from https://akisiel1.github.io/introSparkML/#create-custom-pipeline-stage
  
class FamilySizeTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  # TODO implement a transformer following the instructions from https://akisiel1.github.io/introSparkML/#create-custom-pipeline-stage
  
class IsAloneTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  # TODO implement a transformer following the instructions from https://akisiel1.github.io/introSparkML/#create-custom-pipeline-stage
  
class FarePerPersonTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  # TODO implement a transformer following the instructions from https://akisiel1.github.io/introSparkML/#create-custom-pipeline-stage
  
class DropRowsWithNullTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  # TODO implement a transformer following the instructions from https://akisiel1.github.io/introSparkML/#create-custom-pipeline-stage
  
class CategoryToIndexTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  # TODO implement a transformer following the instructions from https://akisiel1.github.io/introSparkML/#create-custom-pipeline-stage
  
class CategoryToOneHotEncodedTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  # TODO implement a transformer following the instructions from https://akisiel1.github.io/introSparkML/#create-custom-pipeline-stage
  
class DropStringColsTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  # TODO implement a transformer following the instructions from https://akisiel1.github.io/introSparkML/#create-custom-pipeline-stage

<>:11: DeprecationWarning: invalid escape sequence \.
<>:11: DeprecationWarning: invalid escape sequence \.
<command-3057669862061282>:11: DeprecationWarning: invalid escape sequence \.
 _df = df.withColumn('Initial', F.regexp_extract( df['Name'], ('([A-Za-z]+)\.'),1 ) )

In [11]:
data = spark.read.csv("wasbs://data@sparkmltrainig.blob.core.windows.net/train.csv", inferSchema = True, header = True)

train_df, test_df = data.randomSplit([0.9, 0.1], seed=42)

In [12]:
stages = []
# TODO create transformes objects and use it in stage array

# hint: for vector asembler class:
cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'IsAlone', 'FarePerPerson', 'Sex_indexed', 'Initial_indexed', 'Embarked_indexed', 'Sex_encoded', 'Initial_encoded', 'Embarked_encoded']


pipeline = Pipeline(stages=stages)

pipelineModel = pipeline.fit(train_df)

In [13]:
pipelinePath = "/tmp/lr_pipeline_model"
pipelineModel.write().overwrite().save(pipelinePath)

In [14]:
savedPipelineModel = PipelineModel.load(pipelinePath)
pred_df = savedPipelineModel.transform(test_df)

In [15]:
regression_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Survived", metricName="rmse")

rmse = regression_evaluator.evaluate(pred_df)
print(f"RMSE is {rmse}")

RMSE is 0.34149375724898745

In [16]:
# TODO compare if RMSE for both cases is the same